## Answer questions

In [412]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [413]:
import pandas as pd
from src.paths import TRANSFORMED_DATA_DIR, MODELS_DIR, RAW_DATA_DIR

In [414]:
data = pd.read_csv(TRANSFORMED_DATA_DIR / 'transformed_data.csv')

In [415]:
data.shape

(4817, 22)

In [416]:
# Convert date columns to datetime
data['registration_date'] = pd.to_datetime(data['registration_date'])
data['sold_at'] = pd.to_datetime(data['sold_at'])

In [417]:
# Change data types from object to categorical
from src.data import convert_object_columns_to_category

data = convert_object_columns_to_category(data)

In [418]:
from src.data import get_train_test_data

In [419]:
import joblib

In [420]:
features = joblib.load(MODELS_DIR / 'features.pkl')
target = joblib.load(MODELS_DIR / 'target.pkl')

In [421]:
# Print the features
features

['model_key',
 'mileage',
 'engine_power',
 'fuel',
 'paint_color',
 'car_type',
 'feature_1',
 'feature_2',
 'feature_3',
 'feature_4',
 'feature_5',
 'feature_6',
 'feature_7',
 'feature_8',
 'age_in_months_when_sold',
 'month_sold_at',
 'season_sold_at',
 'model_initial']

In [422]:
X, y, X_train, X_test, y_train, y_test = get_train_test_data(data, features, target)

In [423]:
# Load the model
model = joblib.load(MODELS_DIR / 'model.pkl')

In [424]:
# Print the model
model

RandomizedSearchCV(cv=4,
                   estimator=XGBRegressor(base_score=None, booster='gblinear',
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None, device=None,
                                          early_stopping_rounds=None,
                                          enable_categorical=True,
                                          eval_metric=None, feature_types=None,
                                          gamma=None, grow_policy=None,
                                          importance_type=None,
                                          interaction_constraints=None,
                                          learning_...
                                          min_child_weight=None, missing=nan,
                                          monotone_constraints=None,
                                          multi_strategy=None,
                                          n_estimators=None, n_jobs=None,
                                          num_parallel_tree=None,
                                          random_state=None, ...),
                   n_iter=20, n_jobs=-1,
                   param_distributions={'alpha': [0, 0.1, 0.5, 1, 5, 10],
                                        'lambda': [0, 0.1, 0.5, 1, 5, 10],
                                        'n_estimators': [1, 5, 10, 50, 100],
                                        'random_state': [42]},
                   random_state=0, scoring='neg_mean_squared_error', verbose=3)

### Q1

In [425]:
# Read feature importance df from models folder
feature_importance = joblib.load(MODELS_DIR / 'feature_importance_df.pkl')

In [426]:
feature_importance

,feature,importance
6,feature_1,0.197172
13,feature_8,0.177422
8,feature_3,0.131055
11,feature_6,0.130271
9,feature_4,0.119730
7,feature_2,0.055590
15,month_sold_at,0.055252
5,car_type,0.049119
2,engine_power,0.040747
12,feature_7,0.033610


### Q2

As found during the data exploration phase:
- Hybrid and electrical cars are more expensive on average.
- Electrical cars average prices were stable from winter to summer, and were not sold in autumn.
- Diesel and petrol cars had similar average prices, although petrol cars had a drop in average prices starting summer 2018.
- The most expensive car type is, on average, suv, although coupe was most expensive at the start of the year and then dropped below suv also starting in summer.
- Coupe and convertible cars were, on average, more expensive in winter than in summer.
- Vans where more expensive, on average, in spring, summer, and autumn, than in winter.
- Subcompact had generally the lowest average prices.
- Paint color does not seem to generally determine or be associated with the average price, except for color green, which consistently had prices much lower than other colors. Maybe not very popular.
- Orange and white cars were sold for more, on average, during summer than during winter and spring.
- Red cars were the opposite, with lower average prices during summer than during winter and spring.

Find similar observations using estimated price instead of real price

In [427]:
pred = model.predict(X)

In [428]:
data_q2 = data.copy()

In [429]:
data_q2['price'] = pred

In [430]:
from src.plots import plot_avg_target_time_series_by_features

In [431]:
# Load car features
car_features = joblib.load(RAW_DATA_DIR / 'car_features.pkl')

In [432]:
plot_avg_target_time_series_by_features(data_q2, car_features)

In [433]:
# Load small cardinality features
small_cardinality_features = joblib.load(RAW_DATA_DIR / 'small_cardinality_features.pkl')

In [434]:
plot_avg_target_time_series_by_features(data_q2, small_cardinality_features)

In [435]:
from src.questions import ttest_mean_price_difference_between_groups_after_filter

In [436]:
grouping_column = 'season_sold_at'
group_1 = 'winter'
group_2 = 'summer'
for feature in small_cardinality_features:
    for feature_value in data_q2[feature].unique():
        t_stat, p_val = ttest_mean_price_difference_between_groups_after_filter(data_q2, feature, feature_value, grouping_column, group_1, group_2)
        print(f'Feature: {feature}, Feature Value: {feature_value}, T-stat: {t_stat:.4f}, P-value: {p_val:.4f}') 

Feature: fuel, Feature Value: diesel, T-stat: -0.3506, P-value: 0.7259
Feature: fuel, Feature Value: petrol, T-stat: 1.4851, P-value: 0.1418
Feature: fuel, Feature Value: hybrid_petrol, T-stat: nan, P-value: nan
Feature: fuel, Feature Value: electro, T-stat: -0.2808, P-value: 0.8257
Feature: car_type, Feature Value: convertible, T-stat: 1.1506, P-value: 0.2635
Feature: car_type, Feature Value: coupe, T-stat: 2.4230, P-value: 0.0198
Feature: car_type, Feature Value: estate, T-stat: 2.1581, P-value: 0.0312
Feature: car_type, Feature Value: hatchback, T-stat: -0.2483, P-value: 0.8041
Feature: car_type, Feature Value: sedan, T-stat: 4.2045, P-value: 0.0000
Feature: car_type, Feature Value: subcompact, T-stat: 0.8107, P-value: 0.4220
Feature: car_type, Feature Value: suv, T-stat: -1.3686, P-value: 0.1716
Feature: car_type, Feature Value: van, T-stat: -0.9596, P-value: 0.3507
Feature: paint_color, Feature Value: black, T-stat: 1.0496, P-value: 0.2942
Feature: paint_color, Feature Value: grey

### Q3

In [437]:
today_date = '3/1/2024'

In [438]:
data_q3 = data.copy()

In [439]:
# Convert date columns to datetime and mock the sold_at date as today's date
data_q3['registration_date'] = pd.to_datetime(data_q3['registration_date'])
data_q3['sold_at'] = today_date
data_q3['sold_at'] = pd.to_datetime(data_q3['sold_at'])

In [440]:
# Calculate mileage per month
data_q3['mileage_per_month'] = data_q3['mileage'] / data_q3['age_in_months_when_sold']

In [441]:
# Calculate age in month at today date and replace in data_q3
data_q3['age_in_months_when_sold'] = (data_q3['sold_at'].dt.to_period('M') - data_q3['registration_date'].dt.to_period('M')).apply(lambda x: x.n)

In [442]:
# Update estimated mileage at today date
data_q3['mileage'] = data_q3['age_in_months_when_sold'] * data_q3['mileage_per_month']

In [443]:
data_q3[['registration_date', 'sold_at', 'age_in_months_when_sold', 'mileage']].head()

,registration_date,sold_at,age_in_months_when_sold,mileage
0,2012-02-01,2024-03-01,145,286754.859155
1,2016-04-01,2024-03-01,95,60147.954545
2,2012-04-01,2024-03-01,143,374449.585714
3,2014-07-01,2024-03-01,116,345396.744186
4,2014-12-01,2024-03-01,111,269444.175000


In [444]:
X_q3, y_q3, X_train_q3, X_test_q3, y_train_q3, y_test_q3 = get_train_test_data(data_q3, features, target)

In [445]:
# Get estimated prices today
pred_q3 = model.predict(X_q3)

In [446]:
# Add 1 year to the age_in_months_when_sold
X_q3['age_in_months_when_sold'] = X_q3['age_in_months_when_sold'] + 12

In [447]:
# Add 1 year worth of mileage
X_q3['mileage'] = X_q3['mileage'] + 12 * data_q3['mileage_per_month']

In [448]:
# Get estimated prices 1 year later
pred_q3_one_year_later = model.predict(X_q3)

In [449]:
X_q3['price_today'] = pred_q3
X_q3['price_one_year_later'] = pred_q3_one_year_later
X_q3['loss'] = X_q3['price_today'] - X_q3['price_one_year_later'] 

In [450]:
price_today_threshold = 20000
loss_threshold = 2000

In [451]:
# Indentify cars that are candidates for buying
candidate_cars = X_q3[(X_q3['price_today'] >= price_today_threshold) & (X_q3['loss'] <= loss_threshold)]

In [452]:
# Show top 10 candidate cars with lowest loss
candidate_cars.sort_values('loss').head(10)

,model_key,mileage,engine_power,fuel,paint_color,car_type,feature_1,feature_2,feature_3,feature_4,...,feature_6,feature_7,feature_8,age_in_months_when_sold,month_sold_at,season_sold_at,model_initial,price_today,price_one_year_later,loss
4346,X5,3090.600000,183,diesel,black,suv,True,True,False,False,...,False,True,True,101,6,summer,X,27763.000000,26594.935547,1168.064453
3910,X5,9278.857143,155,diesel,black,suv,True,True,False,True,...,True,True,True,92,2,winter,X,25853.708984,24654.685547,1199.023438
4121,X6 M,13241.250000,423,petrol,red,suv,True,True,True,False,...,False,True,True,107,4,spring,X,51272.851562,50063.742188,1209.109375
4705,X5 M,18844.800000,230,diesel,silver,suv,True,True,False,True,...,False,False,True,104,8,summer,X,32011.578125,30777.150391,1234.427734
17,650,39463.484375,270,petrol,grey,convertible,True,False,False,False,...,False,True,False,206,9,autumn,6,22454.470703,21215.478516,1238.992188
37,650,39977.079365,270,petrol,grey,convertible,True,False,False,False,...,False,True,False,206,7,summer,6,22185.066406,20944.976562,1240.089844
2980,525,23582.695652,160,diesel,blue,sedan,True,True,True,False,...,True,True,True,106,4,spring,5,22285.769531,21033.154297,1252.615234
4675,X5,41148.046512,160,diesel,grey,suv,True,True,True,False,...,False,True,True,122,8,summer,X,22540.015625,21236.814453,1303.201172
2664,530,37186.666667,195,diesel,grey,sedan,True,True,True,False,...,True,True,True,100,2,winter,5,25634.736328,24316.292969,1318.443359
3718,X5,47620.666667,160,diesel,grey,suv,True,True,True,False,...,False,True,True,122,1,winter,X,21450.185547,20123.605469,1326.580078


In [453]:
# Find the value counts for model_key in candidate_cars if exists. If not try model key initial.
try:
    model_related_value_counts = candidate_cars['model_key'].value_counts()
except KeyError:
    model_related_value_counts = candidate_cars['model_initial'].value_counts()

In [454]:
model_related_value_counts

model_key
X5                  40
X6                  14
640 Gran Coupé       7
X5 M                 6
M3                   5
                    ..
320                  0
318 Gran Turismo     0
630                  0
635                  0
114                  0
Name: count, Length: 75, dtype: int64

In [455]:
# Describe numerical features of candidate cars
candidate_cars.describe()

,mileage,engine_power,age_in_months_when_sold,month_sold_at,price_today,price_one_year_later,loss
count,125.000000,125.000000,125.000000,125.000000,125.000000,125.000000,125.000000
mean,142382.261885,224.008000,126.112000,5.536000,24417.722656,22766.486328,1651.234375
std,61865.822269,45.368295,18.636253,2.333293,4428.595215,4473.045898,205.684799
min,3090.600000,155.000000,92.000000,1.000000,20011.941406,18233.107422,1168.064453
25%,101884.791667,190.000000,115.000000,4.000000,21521.490234,19860.468750,1511.863281
50%,148566.018868,225.000000,125.000000,6.000000,23051.089844,21334.617188,1660.457031
75%,182529.000000,230.000000,132.000000,8.000000,26114.259766,24484.064453,1801.035156
max,320842.247191,423.000000,206.000000,9.000000,51272.851562,50063.742188,1996.123047


In [456]:
# Show the car with the lowest loss
car_index = candidate_cars.sort_values('loss').index[0]
data_q3.loc[car_index]

maker_key                                  BMW
model_key                                   X5
mileage                                 2723.4
engine_power                               183
registration_date          2016-10-01 00:00:00
fuel                                    diesel
paint_color                              black
car_type                                   suv
feature_1                                 True
feature_2                                 True
feature_3                                False
feature_4                                False
feature_5                                False
feature_6                                False
feature_7                                 True
feature_8                                 True
price                                    49100
sold_at                    2024-03-01 00:00:00
age_in_months_when_sold                     89
month_sold_at                                6
season_sold_at                          summer
model_initial

In [457]:
data.loc[car_index]

maker_key                                  BMW
model_key                                   X5
mileage                                    612
engine_power                               183
registration_date          2016-10-01 00:00:00
fuel                                    diesel
paint_color                              black
car_type                                   suv
feature_1                                 True
feature_2                                 True
feature_3                                False
feature_4                                False
feature_5                                False
feature_6                                False
feature_7                                 True
feature_8                                 True
price                                    49100
sold_at                    2018-06-01 00:00:00
age_in_months_when_sold                     20
month_sold_at                                6
season_sold_at                          summer
model_initial

In [458]:
X_q3.loc[car_index]

model_key                            X5
mileage                          3090.6
engine_power                        183
fuel                             diesel
paint_color                       black
car_type                            suv
feature_1                          True
feature_2                          True
feature_3                         False
feature_4                         False
feature_5                         False
feature_6                         False
feature_7                          True
feature_8                          True
age_in_months_when_sold             101
month_sold_at                         6
season_sold_at                   summer
model_initial                         X
price_today                     27763.0
price_one_year_later       26594.935547
loss                        1168.064453
Name: 4346, dtype: object

### Q3 assumming today is a month after latest sold at date in the data

In [459]:
today_date = data['sold_at'].max()

In [460]:
data_q3 = data.copy()

In [461]:
# Convert date columns to datetime and mock the sold_at date as today's date
data_q3['registration_date'] = pd.to_datetime(data_q3['registration_date'])
data_q3['sold_at'] = today_date
data_q3['sold_at'] = pd.to_datetime(data_q3['sold_at'])

In [462]:
# Calculate mileage per month
data_q3['mileage_per_month'] = data_q3['mileage'] / data_q3['age_in_months_when_sold']

In [463]:
# Calculate age in month at today date and replace in data_q3
data_q3['age_in_months_when_sold'] = (data_q3['sold_at'].dt.to_period('M') - data_q3['registration_date'].dt.to_period('M')).apply(lambda x: x.n)

In [464]:
# Update estimated mileage at today date
data_q3['mileage'] = data_q3['age_in_months_when_sold'] * data_q3['mileage_per_month']

In [465]:
data_q3[['registration_date', 'sold_at', 'age_in_months_when_sold', 'mileage']].head()

,registration_date,sold_at,age_in_months_when_sold,mileage
0,2012-02-01,2018-09-01,79,156231.957746
1,2016-04-01,2018-09-01,29,18360.954545
2,2012-04-01,2018-09-01,77,201626.700000
3,2014-07-01,2018-09-01,50,148877.906977
4,2014-12-01,2018-09-01,45,109234.125000


In [466]:
X_q3, y_q3, X_train_q3, X_test_q3, y_train_q3, y_test_q3 = get_train_test_data(data_q3, features, target)

In [467]:
# Get estimated prices today
pred_q3 = model.predict(X_q3)

In [468]:
# Add 1 year to the age_in_months_when_sold
X_q3['age_in_months_when_sold'] = X_q3['age_in_months_when_sold'] + 12

In [469]:
# Add 1 year worth of mileage
X_q3['mileage'] = X_q3['mileage'] + 12 * data_q3['mileage_per_month']

In [470]:
# Get estimated prices 1 year later
pred_q3_one_year_later = model.predict(X_q3)

In [471]:
X_q3['price_today'] = pred_q3
X_q3['price_one_year_later'] = pred_q3_one_year_later
X_q3['loss'] = X_q3['price_today'] - X_q3['price_one_year_later'] 

In [472]:
# Indentify cars that are candidates for buying
candidate_cars = X_q3[(X_q3['price_today'] >= price_today_threshold) & (X_q3['loss'] <= loss_threshold)]

In [473]:
# Show top 10 candidate cars with lowest loss
candidate_cars.sort_values('loss').head(10)

,model_key,mileage,engine_power,fuel,paint_color,car_type,feature_1,feature_2,feature_3,feature_4,...,feature_6,feature_7,feature_8,age_in_months_when_sold,month_sold_at,season_sold_at,model_initial,price_today,price_one_year_later,loss
2400,420 Gran Coupé,597.333333,120,diesel,blue,hatchback,True,True,False,False,...,True,True,True,64,8,summer,4,22075.599609,20916.906250,1158.693359
4346,X5,1071.000000,183,diesel,black,suv,True,True,False,False,...,False,True,True,35,6,summer,X,34187.351562,33019.285156,1168.066406
3910,X5,2622.285714,155,diesel,black,suv,True,True,False,True,...,True,True,True,26,2,winter,X,32448.328125,31249.308594,1199.019531
4121,X6 M,5073.750000,423,petrol,red,suv,True,True,True,False,...,False,True,True,41,4,spring,X,57922.953125,56713.843750,1209.109375
4705,X5 M,6885.600000,230,diesel,silver,suv,True,True,False,True,...,False,False,True,38,8,summer,X,38800.914062,37566.492188,1234.421875
17,650,26819.843750,270,petrol,grey,convertible,True,False,False,False,...,False,True,False,140,9,autumn,6,29268.939453,28029.945312,1238.994141
778,318,6956.000000,100,diesel,blue,estate,True,True,False,False,...,False,True,False,36,3,spring,3,20028.375000,18788.650391,1239.724609
37,650,27168.888889,270,petrol,grey,convertible,True,False,False,False,...,False,True,False,140,7,summer,6,29005.578125,27765.486328,1240.091797
4353,X3,15729.389831,120,diesel,brown,suv,True,True,True,True,...,False,True,True,74,6,summer,X,23941.283203,22693.041016,1248.242188
2980,525,8899.130435,160,diesel,blue,sedan,True,True,True,False,...,True,True,True,40,4,spring,5,29175.146484,27922.533203,1252.613281


In [474]:
# Find the value counts for model_key in candidate_cars if exists. If not try model key initial.
try:
    model_related_value_counts = candidate_cars['model_key'].value_counts()
except KeyError:
    model_related_value_counts = candidate_cars['model_initial'].value_counts()

In [475]:
model_related_value_counts

model_key
X5                  91
X3                  85
530                 25
520                 25
X6                  25
                    ..
335 Gran Turismo     0
418 Gran Coupé       0
630                  0
116                  0
114                  0
Name: count, Length: 75, dtype: int64

In [476]:
# Describe numerical features of candidate cars
candidate_cars.describe()

,mileage,engine_power,age_in_months_when_sold,month_sold_at,price_today,price_one_year_later,loss
count,488.000000,488.000000,488.000000,488.000000,488.000000,488.000000,488.000000
mean,90390.713651,174.905738,68.715164,5.379098,26083.593750,24358.855469,1724.739746
std,42572.143332,46.780357,21.714756,2.132600,5379.039062,5407.961426,188.662186
min,597.333333,75.000000,26.000000,1.000000,20004.464844,18012.267578,1158.693359
25%,60960.743243,135.000000,55.000000,4.000000,21799.558105,20084.249023,1613.562012
50%,87335.131579,171.500000,65.000000,5.000000,24648.466797,22871.435547,1749.262695
75%,116544.085979,190.000000,79.000000,7.000000,29323.658691,27888.367188,1867.071289
max,239292.452830,423.000000,140.000000,9.000000,57922.953125,56713.843750,1999.609375


In [477]:
# Show the car with the lowest loss
car_index = candidate_cars.sort_values('loss').index[0]
data_q3.loc[car_index]

maker_key                                  BMW
model_key                       420 Gran Coupé
mileage                             485.333333
engine_power                               120
registration_date          2014-05-01 00:00:00
fuel                                    diesel
paint_color                               blue
car_type                             hatchback
feature_1                                 True
feature_2                                 True
feature_3                                False
feature_4                                False
feature_5                                False
feature_6                                 True
feature_7                                 True
feature_8                                 True
price                                    30300
sold_at                    2018-09-01 00:00:00
age_in_months_when_sold                     52
month_sold_at                                8
season_sold_at                          summer
model_initial

In [478]:
data.loc[car_index]

maker_key                                  BMW
model_key                       420 Gran Coupé
mileage                                    476
engine_power                               120
registration_date          2014-05-01 00:00:00
fuel                                    diesel
paint_color                               blue
car_type                             hatchback
feature_1                                 True
feature_2                                 True
feature_3                                False
feature_4                                False
feature_5                                False
feature_6                                 True
feature_7                                 True
feature_8                                 True
price                                    30300
sold_at                    2018-08-01 00:00:00
age_in_months_when_sold                     51
month_sold_at                                8
season_sold_at                          summer
model_initial

In [479]:
X_q3.loc[car_index]

model_key                  420 Gran Coupé
mileage                        597.333333
engine_power                          120
fuel                               diesel
paint_color                          blue
car_type                        hatchback
feature_1                            True
feature_2                            True
feature_3                           False
feature_4                           False
feature_5                           False
feature_6                            True
feature_7                            True
feature_8                            True
age_in_months_when_sold                64
month_sold_at                           8
season_sold_at                     summer
model_initial                           4
price_today                  22075.599609
price_one_year_later          20916.90625
loss                          1158.693359
Name: 2400, dtype: object

### Q4

Check training notebook

### Q5

Add other findings from data exploration